## Rasterio with NetCDF, VSIS3, and earthaccess

In [ ]:
import numpy as np
import rasterio
import rasterio as rio
from common import earthaccess_args
from rasterio.session import AWSSession

In [ ]:
dataset = "gpm_imerg"
filename = earthaccess_args[dataset]["filename"]
bucket = earthaccess_args[dataset]["bucket"]
folder = earthaccess_args[dataset]["folder"]
variable = earthaccess_args[dataset]["variable"]
scale = earthaccess_args[dataset]["scale"]
daac = earthaccess_args[dataset]["daac"]

In [ ]:
def configure_auth():
    import boto3
    import earthaccess

    auth = earthaccess.login()
    s3_credentials = auth.get_s3_credentials(daac)
    session = boto3.Session(
        aws_access_key_id=s3_credentials["accessKeyId"],
        aws_secret_access_key=s3_credentials["secretAccessKey"],
        aws_session_token=s3_credentials["sessionToken"],
        region_name="us-west-2",
    )
    rio_env = rio.Env(
        AWSSession(session),
    )
    rio_env.__enter__()
    return rio_env

In [ ]:
def load_data():
    input_uri = f"{folder}/{filename}"
    src = f"NETCDF:/vsis3/{bucket}/{input_uri}:{variable}"
    with rasterio.open(src) as src_da:
        ma = src_da.read(1, masked=True)
        ma = ma.astype("float32", casting="unsafe")
        np.multiply(ma, src_da.scales[0], out=ma, casting="unsafe")
        np.add(ma, src_da.offsets[0], out=ma, casting="unsafe")
    return ma.filled(fill_value=np.nan)

In [ ]:
if __name__ == "__main__":
    rio_env = configure_auth()
    da = load_data()
    rio_env.__exit__()